# Text Classification with Magnitude & ELMo

In [3]:
import numpy as np
import pandas as pd
import tensorflow as tf
import pickle

from sklearn.preprocessing import MultiLabelBinarizer
from keras.layers import Input, Dense
from keras.models import Model
from keras.callbacks import TensorBoard
from pymagnitude import Magnitude
from keras.utils import Sequence

print(tf.__version__)

Using TensorFlow backend.


1.13.0-rc2


In [2]:
# Download the data from GCS

!wget 'https://storage.googleapis.com/movies_data/movies_metadata.csv'

In [3]:
data = pd.read_csv('movies_metadata.csv')
data.head()

/Users/smap3/.local/share/virtualenvs/tensorflow_hub_vs_magnitude-pY8tZzeD/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3044: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [4]:
# urllib.request.urlretrieve('https://storage.googleapis.com/bq-imports/descriptions.p', 'descriptions.p')
# urllib.request.urlretrieve('https://storage.googleapis.com/bq-imports/genres.p', 'genres.p')

descriptions = pickle.load(open('pickle/descriptions.p', 'rb'))
genres = pickle.load(open('pickle/genres.p', 'rb'))



In [5]:
train_size = int(len(descriptions) * .8)

train_descriptions = descriptions[:train_size].astype('str')
train_genres = genres[:train_size]

test_descriptions = descriptions[train_size:].astype('str')
test_genres = genres[train_size:]

In [6]:
encoder = MultiLabelBinarizer()
encoder.fit_transform(train_genres)
train_encoded = encoder.transform(train_genres)
test_encoded = encoder.transform(test_genres)
num_classes = len(encoder.classes_)

# Print all possible genres and the labels for the first movie in our training dataset
print(encoder.classes_)
print(train_descriptions.values[0])
print(train_encoded[0])


['Action' 'Adventure' 'Comedy' 'Crime' 'Documentary' 'Horror' 'Romance'
 'Science Fiction' 'Thriller']
A family wedding reignites the ancient feud between next-door neighbors and fishing buddies John and Max. Meanwhile, a sultry Italian divorcée opens a restaurant at the local bait shop, alarming the locals who worry she'll scare the fish away. But she's less interested in seafood than she is in cooking up a hot time with Max.
[0 0 1 0 0 0 1 0 0]


In [5]:
%%time
# Need to download Magnitude object before execute. 
# http://magnitude.plasticity.ai/elmo/medium/elmo_2x4096_512_2048cnn_2xhighway_5.5B_weights.magnitude

elmo_vecs = Magnitude('./models/elmo_2x4096_512_2048cnn_2xhighway_5.5B_weights.magnitude')

CPU times: user 62.4 ms, sys: 64.8 ms, total: 127 ms
Wall time: 114 ms


In [7]:
class BatchSequence(Sequence):
    
    def __init__(self, x_set, y_set, batch_size, shuffle=True):
        data_size = len(x_set)

        if shuffle:
            shuffle_indices = np.random.permutation(np.arange(data_size))
            self.x = x_set[shuffle_indices]
            self.y = y_set[shuffle_indices]
        else:
            self.x, self.y = x_set, y_set
            
        self.batch_size = batch_size

    def __len__(self):
        return int(np.ceil(len(self.x) / float(self.batch_size)))

    def __getitem__(self, idx):
        start_index = idx * self.batch_size
        end_index = min((idx + 1) * self.batch_size, len(self.x))
        
        batch_x = self.x[start_index: end_index]
        batch_y = self.y[start_index: end_index]

        splited_data = list(map(lambda x: x.split(' '), batch_x))
        query_vecs = elmo_vecs.query(splited_data)
        embeddings = np.mean(elmo_vecs.unroll(query_vecs), axis=(1, 2))
        
        X, y = np.array(embeddings), np.array(batch_y)

        return X, y


In [8]:
# elmo embedding dimension
elmo_dim = 1024

# Input Layers
word_input = Input(shape=(elmo_dim, ), dtype=tf.float32)  # (batch_size, sent_length)

# Hidden Layers
x = Dense(64, activation='relu')(word_input)

# Output Layer
predict = Dense(units=9, activation='sigmoid')(x)

model = Model(inputs=[word_input], outputs=predict)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc'])

model.summary()

# plot_model(model, to_file="model.png", show_shapes=True)



Instructions for updating:
Colocations handled automatically by placer.


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 1024)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 64)                65600     
_________________________________________________________________
dense_2 (Dense)              (None, 9)                 585       
Total params: 66,185
Trainable params: 66,185
Non-trainable params: 0
_________________________________________________________________


In [12]:
# Create an instance of batch sequence
batch_size = 32

train_batchSequence = BatchSequence(train_descriptions.values, train_encoded, batch_size)
valid_batchSequence = BatchSequence(test_descriptions.values, test_encoded, batch_size)


In [13]:
# Train model

logfile_path = './log'
tb_cb = TensorBoard(log_dir=logfile_path, histogram_freq=0)

history = model.fit_generator(train_batchSequence,
                              epochs=5, 
                              validation_data=valid_batchSequence,
                              callbacks=[tb_cb])

Epoch 1/5


  1/375 [..............................] - ETA: 3:17:37 - loss: 0.2483 - acc: 0.8924

  2/375 [..............................] - ETA: 2:58:26 - loss: 0.2659 - acc: 0.8889

  3/375 [..............................] - ETA: 2:38:54 - loss: 0.2771 - acc: 0.8819

  4/375 [..............................] - ETA: 2:44:00 - loss: 0.2772 - acc: 0.8819

...

371/375 [============================>.] - ETA: 1s - loss: 0.2385 - acc: 0.9001

372/375 [============================>.] - ETA: 1s - loss: 0.2384 - acc: 0.9001

373/375 [============================>.] - ETA: 0s - loss: 0.2384 - acc: 0.9001

374/375 [============================>.] - ETA: 0s - loss: 0.2385 - acc: 0.9001

375/375 [==============================] - 163s 436ms/step - loss: 0.2385 - acc: 0.9001 - val_loss: 0.2325 - val_acc: 0.9082


In [15]:
# Evaluate model

score = model.evaluate_generator(valid_batchSequence)
print('Test score:', score[0])
print('Test accuracy;', score[1])


Test score: 0.2324758847282902
Test accuracy; 0.9082286627183719
